In [15]:
import numpy as np
import random
import math
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from vmdpy import VMD
from scipy.fftpack import hilbert,fft,ifft
from math import log
import pandas as pd
import time

In [16]:
# data = np.loadtxt('ball18.txt')
# x = data[0:2048]
df = pd.read_csv('solar2019.csv',header=0, parse_dates=[0],index_col=0)# 读取CSV文件，并设置第一列为日期，作为索引
# print(df.head(10))
x = df.iloc[0:2000,-1].values
print(x.shape)
tau = 0.  # noise-tolerance (no strict fidelity enforcement)
DC = 0  # no DC part imposed
init = 1  # initialize omegas uniformly
tol = 1e-7

(2000,)


In [17]:
def training(X):
    K = int(X[0])  # 将X中的第一个元素转换为整数，并赋值给neurons1
    alpha = int(X[1])  # 将X中的第二个元素转换为整数，并赋值给neurons2
    if K < int(X[0]):
        K = int(X[0])
    if K > int(X[0]):
        K = int(X[0])
        
    if alpha <int(X[1]):
        alpha = int(X[1])
    if alpha > int(X[1]):
        alpha =int(X[1])
    print([K,alpha])  # 打印神经元数量、dropout和batch_size

    u, u_hat, omega = VMD(x, alpha, tau, K, DC, init, tol)

    EP = []
    for i in range(K):
        H = np.abs(hilbert(u[i,:]))
        e1 = []
        for j in range(len(H)):
            p = H[j]/np.sum(H)
            e = -p*log(p,2)
            e1.append(e)
        E = np.sum(e1)  
        EP.append(E)
    s = np.sum(EP)/K
    return s  # 返回均方误差作为训练函数的输出






# def fun(K, alpha):
#     if K < lb[0]:
#         K = lb[0]
#     if K > ub[0]:
#         K = ub[0]
        
#     if alpha < lb[1]:
#         alpha = lb[1]
#     if alpha > ub[1]:
#         alpha =ub[1]

#     # K = int(position[0])
#     # alpha = position[1]
#     u, u_hat, omega = VMD(x, alpha, tau, K, DC, init, tol)
#     #  
#     EP = []
#     for i in range(K):
#         H = np.abs(hilbert(u[i,:]))
#         e1 = []
#         for j in range(len(H)):
#             p = H[j]/np.sum(H)
#             e = -p*log(p,2)
#             e1.append(e)
#         E = np.sum(e1)  
#         EP.append(E)
#     s = np.sum(EP)/K
#     return s

In [18]:
class SSA():
    def __init__(self, func, n_dim=None, pop_size=20, max_iter=10, lb=-512, ub=512, verbose=False):
        self.func = func
        self.n_dim = n_dim  # 粒子的维度，也是目标函数的变量个数
        self.pop = pop_size  # 粒子数量
        P_percent = 0.2  # # 生产者的人口规模占总人口规模的20%
        D_percent = 0.1  # 预警者的人口规模占总人口规模的10%
        self.pNum = round(self.pop * P_percent)  # 生产者的人口规模占总人口规模的20%
        self.warn = round(self.pop * D_percent)  # 预警者的人口规模占总人口规模的10%

        self.max_iter = max_iter  # 最大迭代次数
        self.verbose = verbose # 是否打印每次迭代的结果
        
        # 根据给定的 lb 和 ub 列表设置下界和上界数组
        self.lb, self.ub = np.array(lb) * np.ones(self.n_dim), np.array(ub) * np.ones(self.n_dim)
        # 检查维度是否等于 lb 和 ub 数组的长度
        assert self.n_dim == len(self.lb) == len(self.ub), 'dim == len(lb) == len(ub) is not True'
        # 检查所有的上界元素是否大于对应的下界元素
        assert np.all(self.ub > self.lb), 'upper-bound must be greater than lower-bound'
        # 从下界（self.lb）到上界（self.ub）之间生成服从均匀分布的随机数矩阵
        self.X = np.random.uniform(low=self.lb, high=self.ub, size=(self.pop, self.n_dim))

        self.Y = [self.func(self.X[i]) for i in range(len(self.X))]  ## 对于每个粒子，计算其函数值（y = f(x)）
        self.pbest_x = self.X.copy() # 每个粒子的个体最佳位置（历史上的最佳位置）
        self.pbest_y = [np.inf for i in range(self.pop)]  # 每个粒子的最佳函数值（历史上的最佳函数值）
        self.gbest_x = self.pbest_x.mean(axis=0).reshape(1, -1)  # 所有粒子的全局最佳位置
        self.gbest_y = np.inf  # 所有粒子的全局最佳函数值
        self.gbest_y_hist = []  # 存储每次迭代的全局最佳函数值
        self.update_pbest() # 更新每个粒子的个体最佳位置和函数值
        self.update_gbest() # 更新全局最佳位置和函数值
        #
        # 记录详细值
        self.record_mode = False
        self.record_value = {'X': [], 'V': [], 'Y': []}
        self.best_x, self.best_y = self.gbest_x, self.gbest_y  # 为了历史原因，记录最佳位置和函数值（即全局最佳位置和函数值）
        # 初始化最大函数值的索引、粒子位置和函数值
        self.idx_max = 0
        self.x_max = self.X[self.idx_max, :]
        self.y_max = self.Y[self.idx_max]

    def cal_y(self, start, end): # 计算从索引 start 到索引 end-1 的每个粒子的函数值 Y
        # 对于每个索引 i 在 start 和 end-1 之间
        for i in range(start, end):
            self.Y[i] = self.func(self.X[i])# 计算粒子 X[i] 的函数值并存储在 Y[i] 中
        # return self.Y

    def update_pbest(self): # 更新每个个体最佳位置（pbest）
        '''
        个体最佳位置（pbest）
        '''
        for i in range(len(self.Y)):  # 对于每个索引 i 在 0 到 len(self.Y)-1 之间
            # 如果当前粒子的函数值 self.Y[i] 比个体最佳函数值 self.pbest_y[i] 更优
            if self.pbest_y[i] > self.Y[i]:
                # 更新该粒子的个体最佳位置和函数值
                self.pbest_x[i] = self.X[i]
                self.pbest_y[i] = self.Y[i]

    def update_gbest(self): # 更新全局最佳位置（gbest）
        # 找到当前个体最佳函数值中最小值的索引
        idx_min = self.pbest_y.index(min(self.pbest_y))
        # 如果全局最佳函数值 self.gbest_y 比当前个体最佳函数值中的最小值更优
        if self.gbest_y > self.pbest_y[idx_min]:
            # 更新全局最佳位置和函数值
            self.gbest_x = self.X[idx_min, :].copy()
            self.gbest_y = self.pbest_y[idx_min]

    def find_worst(self): # 找到当前函数值中的最大值（用于记录最差的个体）
        # 找到当前函数值 self.Y 中的最大值的索引
        self.idx_max = self.Y.index(max(self.Y))
        # 获取对应最大值索引的粒子位置和函数值
        self.x_max = self.X[self.idx_max, :]
        self.y_max = self.Y[self.idx_max]

    def update_finder(self):
        r2 = np.random.rand(1)  # 预警值
        self.idx = sorted(enumerate(self.Y), key=lambda x: x[1])
        self.idx = [self.idx[i][0] for i in range(len(self.idx))]
        # 这一部位为发现者（探索者）的位置更新
        if r2 < 0.8:  # 预警值较小，说明没有捕食者出现
            for i in range(self.pNum):
                r1 = np.random.rand(1)
                self.X[self.idx[i], :] = self.X[self.idx[i], :] * np.exp(-(i) / (r1 * self.max_iter))  # 对自变量做一个随机变换
                self.X = np.clip(self.X, self.lb, self.ub)  # 对超过边界的变量进行去除
                # X[idx[i], :] = Bounds(X[idx[i], :], lb, ub)  # 对超过边界的变量进行去除
                # fit[sortIndex[0, i], 0] = func(X[sortIndex[0, i], :])  # 算新的适应度值
        elif r2 >= 0.8:  # 预警值较大，说明有捕食者出现威胁到了种群的安全，需要去其它地方觅食
            for i in range(self.pNum):
                Q = np.random.rand(1)  # 也可以替换成  np.random.normal(loc=0, scale=1.0, size=1)
                self.X[self.idx[i], :] = self.X[self.idx[i], :] + Q * np.ones(
                    (1, self.n_dim))  # Q是服从正态分布的随机数。L表示一个1×d的矩阵
                self.X = np.clip(self.X, self.lb, self.ub)  # 对超过边界的变量进行去除
                # X[idx[i], :] = Bounds(X[sortIndex[0, i], :], lb, ub)
                # fit[sortIndex[0, i], 0] = func(X[sortIndex[0, i], :])
        self.cal_y(0, self.pNum)

    def update_follower(self): 
        #  这一部位为加入者（追随者）的位置更新
        for ii in range(self.pop - self.pNum):  # 对于每个加入者的索引 ii 在范围 self.pNum 到 self.pop - 1 之间
            i = ii + self.pNum  # 计算当前加入者的索引 i
            A = np.floor(np.random.rand(1, self.n_dim) * 2) * 2 - 1 # 生成二值随机矩阵 A
            best_idx = self.Y[0:self.pNum].index(min(self.Y[0:self.pNum])) # 找到个体最佳函数值中的最小值的索引
            bestXX = self.X[best_idx, :] # 获取个体最佳位置
            if i > self.pop / 2:  # 如果 i 大于 pop 的一半
                Q = np.random.rand(1) # 生成随机数 Q
                # 根据公式更新加入者的位置
                self.X[self.idx[i], :] = Q * np.exp((self.x_max - self.X[self.idx[i], :]) / np.square(i))
            else:
                # 根据公式更新加入者的位置
                self.X[self.idx[i], :] = bestXX + np.dot(np.abs(self.X[self.idx[i], :] - bestXX),
                                                         1 / (A.T * np.dot(A, A.T))) * np.ones((1, self.n_dim))
        self.X = np.clip(self.X, self.lb, self.ub)  # 对超过边界的变量进行去除
        # X[self.idx[i],:] = Bounds(X[self.idx[i],lb,ub)
        # fit[self.idx[i],0] = func(X[self.idx[i], :])
        self.cal_y(self.pNum, self.pop) # 计算加入者的函数值

    def detect(self): # 检测是否需要进行位置更新
        arrc = np.arange(self.pop) # 创建一个包含索引的数组 arrc
        c = np.random.permutation(arrc)  # 随机排列序列
        b = [self.idx[i] for i in c[0: self.warn]] # 选取排列后的前 self.warn 个索引作为 b
        e = 10e-10 # 定义一个很小的数 e
        # 对于 b 中的每个索引 j
        for j in range(len(b)):
            # 如果当前粒子的函数值 self.Y[b[j]] 大于全局最佳函数值 self.gbest_y
            if self.Y[b[j]] > self.gbest_y:
                # 更新粒子的位置为全局最佳函数值加上一个随机向量
                self.X[b[j], :] = self.gbest_y + np.random.rand(1, self.n_dim) * np.abs(self.X[b[j], :] - self.gbest_y)
            else:
                # 更新粒子的位置为原位置加上一个随机向量
                self.X[b[j], :] = self.X[b[j], :] + (2 * np.random.rand(1) - 1) * np.abs(
                    self.X[b[j], :] - self.x_max) / (self.func(self.X[b[j]]) - self.y_max + e)
            # X[sortIndex[0, b[j]], :] = Bounds(X[sortIndex[0, b[j]], :], lb, ub)
            # fit[sortIndex[0, b[j]], 0] = func(X[sortIndex[0, b[j]]])
            self.X = np.clip(self.X, self.lb, self.ub)  # 对超过边界的变量进行去除
            self.Y[b[j]] = self.func(self.X[b[j]])

    def run(self, max_iter=None):
        self.max_iter = max_iter or self.max_iter
        for iter_num in range(self.max_iter): # 对于每次迭代 iter_num 在范围 0 到 self.max_iter-1 之间
            self.update_finder()  # 更新发现者位置
            self.find_worst()  # 取出最大的适应度值和最差适应度的X
            self.update_follower()  # 更新跟随着位置
            self.update_pbest()
            self.update_gbest()
            self.detect()
            self.update_pbest()
            self.update_gbest()
            self.gbest_y_hist.append(self.gbest_y) # 记录全局最佳函数值的历史记录
            # 打印每次迭代的函数值
            print(f"Iteration {iter_num + 1}: {self.gbest_y}")
        return self.best_x, self.best_y # 返回历史上的最佳位置和最佳函数值

In [19]:
# 定义参数的上限
UP = [10, 3000]
# 定义参数的下限
DOWN = [2, 100]
start_time = time.time()  # 记录开始时间
# 创建 SSA 对象，设置参数并进行优化
ssa = SSA(training, n_dim=2, pop_size=20, max_iter=40, lb=DOWN, ub=UP)
ssa.run()
end_time = time.time()  # 记录结束时间
execution_time = end_time - start_time  # 计算代码执行时间
print("代码执行时间：", execution_time, "秒")
# 打印最优参数
print('best_params is ', ssa.gbest_x)
# 打印最优精度
print('best_precision is',ssa.gbest_y)

[6, 249]
[5, 969]
[6, 2950]
[8, 2474]
[7, 1524]
[8, 2248]
[7, 1203]
[2, 1421]
[2, 2575]
[9, 2976]
[9, 2678]
[8, 612]
[6, 2007]
[4, 1895]
[6, 632]
[3, 2595]
[6, 2190]
[8, 1539]
[5, 1398]
[4, 1421]
[7, 249]
[5, 969]
[6, 2950]
[8, 2474]
[2, 100]
[10, 1248]
[10, 726]
[2, 198]
[2, 100]
[2, 100]
[2, 100]
[8, 612]
[2, 100]
[2, 100]
[6, 632]
[2, 100]
[2, 100]
[8, 1540]
[2, 100]
[2, 100]
[10, 100]
[10, 100]
Iteration 1: 9.932501520378349
[5, 210]
[2, 2487]
[2, 100]
[2, 100]
[2, 100]
[10, 731]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[8, 576]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[10, 100]
[10, 874]
[9, 100]
[2, 100]
[10, 100]
[5, 210]
[2, 150]
Iteration 2: 9.882829733094658
[2, 150]
[2, 100]
[10, 267]
[2, 153]
[10, 355]
[10, 499]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[7, 499]
[2, 100]
[2, 100]
[2, 100]
[10, 100]
[9, 100]
[2, 100]
[5, 100]
[2, 100]
[10, 100]
[9, 100]
Iteration 3: 9.881598277885232
[10, 330]
[10, 355]
[10, 267]
[2, 214]
[10, 311]
[2, 151]
[8, 100]
[2, 100]
[2, 100]
[2, 1

[5, 100]
[5, 100]
[2, 100]
[5, 100]
[5, 100]
[6, 100]
[2, 100]
[3, 100]
[2, 100]
[2, 100]
[2, 100]
[5, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[4, 100]
[2, 100]
[10, 100]
[10, 100]
Iteration 35: 9.881598277885232
[6, 101]
[5, 100]
[5, 100]
[2, 100]
[4, 100]
[4, 100]
[6, 100]
[2, 100]
[4, 100]
[2, 100]
[2, 100]
[2, 100]
[5, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[4, 100]
[7, 102]
[10, 100]
[10, 100]
Iteration 36: 9.881598277885232
[6, 101]
[5, 100]
[4, 100]
[2, 100]
[4, 100]
[5, 100]
[4, 100]
[2, 100]
[5, 100]
[2, 100]
[2, 100]
[2, 100]
[4, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[9, 100]
[7, 102]
[10, 100]
[10, 100]
Iteration 37: 9.881598277885232
[5, 100]
[5, 100]
[4, 100]
[3, 100]
[5, 100]
[5, 100]
[5, 100]
[2, 100]
[4, 100]
[2, 100]
[5, 100]
[2, 100]
[5, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[2, 100]
[7, 102]
[2, 100]
[5, 100]
[2, 100]
[10, 100]
Iteration 38: 9.881598277885232
[5, 100]
[4, 100]
[4, 100]
[2, 100]
[4, 100]
[3, 100]
[5, 100]
[2, 100]
[

In [38]:
import numpy as np
from numpy.random import random as rand
import random
import math
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from vmdpy import VMD
from scipy.fftpack import hilbert,fft,ifft
from math import log
import pandas as pd


df = pd.read_csv('solar2019.csv',header=0, parse_dates=[0],index_col=0)# 读取CSV文件，并设置第一列为日期，作为索引
# print(df.head(10))
x = df.iloc[0:2000,-1].values


tau = 0.  # noise-tolerance (no strict fidelity enforcement)
DC = 0  # no DC part imposed
init = 1  # initialize omegas uniformly
tol = 1e-7

#========参数设置==============
# objfun:目标函数
# N_pop: 种群规模，通常为10到40
# N_gen: 迭代数
# A: 响度（恒定或降低）
# r: 脉冲率（恒定或减小）
# 此频率范围决定范围
# 如有必要，应更改这些值
# Qmin: 频率最小值
# Qmax: 频率最大值
# d: 维度
# lower: 下界
# upper: 上界

def bat_algorithm(objfun, N_pop=10, N_gen=4, A=0.5, r=0.5,
    Qmin=0, Qmax=2, d=2, lower=[3,100], upper=[10,3000]):

    N_iter = 0

    # 速度上下限
    Lower_bound = lower * np.ones((1,d))
    Upper_bound = upper * np.ones((1,d))

    Q = np.zeros((N_pop, 1)) # 频率
    v = np.zeros((N_pop, d)) # 速度
    S = np.zeros((N_pop, d))

    # 初始化种群、初始解

    Sol = np.zeros((N_pop, d))
    Fitness = np.zeros((N_pop, 1))
    for i in range(N_pop):
        Sol[i] = np.random.uniform(Lower_bound, Upper_bound, (1, d))
        Fitness[i] = objfun(Sol[i])

    # 找出初始最优解
    fmin = min(Fitness)
    Index = list(Fitness).index(fmin)
    best = Sol[Index]

    # 迭代
    for t in range(N_gen):
        print('迭代次数：',t+1)

        for i in range(N_pop):

            Q[i] = np.random.uniform(Qmin, Qmax)
            v[i] = v[i] + (Sol[i] - best) * Q[i]
            S[i] = Sol[i] + v[i]

            # 界限限制
            Sol[i] = simplebounds(Sol[i], Lower_bound, Upper_bound)
            # Pulse rate
            if rand() > r:

                S[i] = best + 0.001*np.random.randn(1, d)

            #====评估新的解决方案 ===========
            # print(i)
            Fnew = objfun(S[i])
            #====如果解决方案有所改进，或者声音不太大，请更新====
            if (Fnew <= Fitness[i]) and (rand() < A):
                Sol[i] = S[i]
                Fitness[i] = Fnew

            #====更新当前的最佳解决方案======
            if Fnew <= fmin:
                best = S[i]
                fmin = Fnew

        N_iter = N_iter + N_pop

    print('Number of evaluations: ', N_iter)
    print("Best = ", int(best[0]),best[1], '\n fmin = ', fmin)

    return best


def simplebounds(s, Lower_bound, Upper_bound):

    Index = s > Lower_bound
    s = Index * s + ~Index * Lower_bound
    Index = s < Upper_bound
    s = Index * s + ~Index * Upper_bound

    return s


#====目标函数=============
def test_function(u):
    low = [3,100]
    ub = [10,3000]
    K = int(u[0])
    alpha = u[1]
    if K < low[0]:
        K = low[0]
    if K > ub[0]:
        K = ub[0]

    if alpha < low[1]:
        alpha = low[1]
    if alpha > ub[1]:
        alpha =ub[1]


    u, u_hat, omega = VMD(x, alpha, tau, K, DC, init, tol)
    #
    EP = []
    for i in range(K):
        H = np.abs(hilbert(u[i,:]))
        e1 = []
        for j in range(len(H)):
            p = H[j]/np.sum(H)
            e = -p*log(p,2)
            e1.append(e)
        E = np.sum(e1)
        EP.append(E)
    s = np.sum(EP)/K
    return s


if __name__ == '__main__':
    print(bat_algorithm(test_function))
    bat_algorithm(test_function)

迭代次数： 1
迭代次数： 2
迭代次数： 3
迭代次数： 4
Number of evaluations:  40
Best =  6 -2144.6085258688863 
 fmin =  9.956738005023835
[    6.31682999 -2144.60852587]
迭代次数： 1
迭代次数： 2
迭代次数： 3
迭代次数： 4
Number of evaluations:  40
Best =  9 588.6268530411858 
 fmin =  10.101025406741659


In [3]:
##GA优化

# from sklearn import svm
# import numpy as np
# from numpy.random import random as rand
# import random
# import math
# from matplotlib import pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# from vmdpy import VMD
# from scipy.fftpack import hilbert,fft,ifft
# from math import log
# import pandas as pd
# import copy


# df = pd.read_csv('solar2019.csv',header=0, parse_dates=[0],index_col=0)# 读取CSV文件，并设置第一列为日期，作为索引
# # print(df.head(10))
# x = df.iloc[0:2000,-1].values


# tau = 0.  # noise-tolerance (no strict fidelity enforcement)
# DC = 0  # no DC part imposed
# init = 1  # initialize omegas uniformly
# tol = 1e-7
 


# def Fun(vardim, x, bound):
    
#     K = int(x[0])
#     alpha = x[1]
    
#     if K < low[0]:
#         K = low[0]
#     if K > ub[0]:
#         K = ub[0]
        
#     if alpha < low[1]:
#         alpha = low[1]
#     if alpha > ub[1]:
#         alpha =ub[1]  

    
#     u, u_hat, omega = VMD(x, alpha, tau, K, DC, init, tol)
#     #  平均包络熵
#     EP = []
#     for i in range(K):
#         H = np.abs(hilbert(u[i,:]))
#         e1 = []
#         for j in range(len(H)):
#             p = H[j]/np.sum(H)
#             e = -p*log(p,2)
#             e1.append(e)
#         E = np.sum(e1)  
#         EP.append(E)
#     s = np.sum(EP)/K
#     return s
    


# class GAIndividual:
#     '''
    
#     创建pop中的单个个体
#     '''
#     def __init__(self, vardim, bound):
        
#         self.vardim = vardim
#         self.bound = bound
#         self.fitness = 0

#     def generate(self):
#         '''
#         generate a random chromsome for genetic algorithm
#         '''
#         len = self.vardim
#         rnd = np.random.random(size=len)
#         self.chrom = np.zeros(len)
#         for i in range(0, len):
#             self.chrom[i] = self.bound[0][i] + \
#                 (self.bound[1][i] - self.bound[0][i]) * rnd[i]
#         print(int(self.chrom[0]),self.chrom[1])

#     def calculateFitness(self):
#         '''
#         calculate the fitness of the chromsome
#         '''
#         self.fitness = Fun(self.vardim, self.chrom, self.bound)
#         print(self.fitness)
        

# class GeneticAlgorithm:
    
#     def __init__(self, sizepop, vardim, bound, MAXGEN, params):
#         '''
#         sizepop: population sizepop
#         vardim: dimension of variables
#         bound: boundaries of variables
#         MAXGEN: termination condition
#         param: algorithm required parameters, it is a list which is consisting 
#                of crossover rate, mutation rate, alpha
#         '''
#         self.sizepop = sizepop
#         self.MAXGEN = MAXGEN
#         self.vardim = vardim
#         self.bound = bound
#         self.population = []
#         self.fitness = np.zeros((self.sizepop, 1))
#         self.trace = np.zeros((self.MAXGEN, 2))
#         self.params = params

#     def initialize(self):
        
#         for i in range(0, self.sizepop):
#             ind = GAIndividual(self.vardim, self.bound)
#             ind.generate()
#             self.population.append(ind)

#     def evaluate(self):
        
#         for i in range(0, self.sizepop):
#             self.population[i].calculateFitness()
#             self.fitness[i] = self.population[i].fitness

#     def solve(self):
        
#         self.t = 0 # 迭代次数
#         self.initialize() # 初始化种群
#         self.evaluate() # 计算适应度
#         best = np.min(self.fitness) #选出适应度最小的个体
#         print("当前最小适应度：",best)
#         bestIndex = np.argmin(self.fitness) # 最小适应度的索引
#         self.best = copy.deepcopy(self.population[bestIndex]) 
#         self.avefitness = np.mean(self.fitness) # 平均适应度
#         self.BEST = []
#         while (self.t < self.MAXGEN):
#             print('迭代次数：',self.t)
#             self.t += 1
#             self.selectionOperation() # 选择
#             self.crossoverOperation() # 交叉
#             self.mutationOperation()  # 变异
#             self.evaluate()           # 重新计算新种群适应度
#             best = np.min(self.fitness)
#             bestIndex = np.argmin(self.fitness)
#             if best < self.best.fitness:
#                 self.best = copy.deepcopy(self.population[bestIndex])
#             self.avefitness = np.mean(self.fitness)
#             self.BEST.append(self.best)
           
#         print("Optimal solution is:", int(self.best.chrom[0]),self.best.chrom[1])
        

#     def selectionOperation(self):
#         '''
#         selection operation for Genetic Algorithm
#         '''
#         newpop = []
#         totalFitness = np.sum(self.fitness)
#         accuFitness = np.zeros((self.sizepop, 1))

#         # 适应度的累进占比
#         sum1 = 0.
#         for i in range(0, self.sizepop):
#             accuFitness[i] = sum1 + self.fitness[i] / totalFitness
#             sum1 = accuFitness[i]

#         # 随机选出新种群的索引
#         for i in range(0, self.sizepop):
#             r = random.random()
#             idx = 0
#             for j in range(0, self.sizepop - 1):
#                 if j == 0 and r < accuFitness[j]:
#                     idx = 0
#                     break
#                 elif r >= accuFitness[j] and r < accuFitness[j + 1]:
#                     idx = j + 1
#                     break
#             newpop.append(self.population[idx])
#         self.population = newpop

#     def crossoverOperation(self):
#         '''
#         crossover operation for genetic algorithm
#         '''
#         newpop = []
#         # 选出两个个体进行交换
#         for i in range(0, self.sizepop, 2):
#             idx1 = random.randint(0, self.sizepop - 1)
#             idx2 = random.randint(0, self.sizepop - 1)
#             while idx2 == idx1:
#                 idx2 = random.randint(0, self.sizepop - 1)
#             newpop.append(copy.deepcopy(self.population[idx1]))
#             newpop.append(copy.deepcopy(self.population[idx2]))
#             r = random.random()
            
#             if r < self.params[0]:
#                 crossPos = random.randint(1, self.vardim - 1)
#                 for j in range(crossPos, self.vardim):
                   
#                     newpop[i].chrom[j] = newpop[i].chrom[j] * self.params[2] +\
#                         (1 - self.params[2]) * newpop[i + 1].chrom[j]
                    
#                     newpop[i + 1].chrom[j] = newpop[i + 1].chrom[j] * self.params[2] + \
#                         (1 - self.params[2]) * newpop[i].chrom[j]
#         self.population = newpop

#     def mutationOperation(self):
#         '''
#         mutation operation for genetic algorithm
#         '''
#         newpop = []
#         for i in range(0, self.sizepop):
#             newpop.append(copy.deepcopy(self.population[i]))
#             r = random.random()
#             if r < self.params[1]:
#                 mutatePos = random.randint(0, self.vardim - 1)
#                 theta = random.random()
#                 if theta > 0.5:
                    
#                     newpop[i].chrom[mutatePos] = newpop[i].chrom[mutatePos] - \
#                         (newpop[i].chrom[mutatePos] - self.bound[0][mutatePos]) * \
#                             (1 - random.random()**(1 - self.t / self.MAXGEN))
#                 else:
                   
#                     newpop[i].chrom[mutatePos] = newpop[i].chrom[mutatePos] + \
#                         (self.bound[1][mutatePos] - newpop[i].chrom[mutatePos]) * \
#                             (1 - random.random()**(1 - self.t / self.MAXGEN))
#         self.population = newpop


# if __name__ == "__main__":
#     low = [3,100]
#     ub = [10, 3000]
#     bound = [low,ub]
#     print(bound)
#     # def __init__(self, sizepop, vardim, bound, MAXGEN, params):
#     ga = GeneticAlgorithm(60, 2, bound, 40, [0.9, 0.1, 0.5])
#     ga.solve()

[[3, 100], [10, 3000]]
6 2406.023117363554
5 2764.019315752946
9 1109.4121163773411
3 254.2201598049656
4 1947.5801994178626
7 2072.845961972953
5 1270.962380972338
7 847.5727103299198
5 1040.4299964320464
4 2413.3257725332674
3 1593.4648917345978
4 2166.278349908661
8 2652.2775719854144
4 2948.202438307074
7 1229.7124323570522
9 1271.2385954281285
5 2610.705428587702
3 504.3805725774293
3 1500.1262734945883
6 2402.540978502412
7 2256.620619802535
4 657.2442281798837
4 1724.9557785522693
6 1959.386638574193
9 234.4537640431314
5 1487.371453259424
4 2445.4796865047483
7 1415.5906813789124
8 1806.986036643306
9 2491.6485199573253
6 1300.6119429083244
4 1776.807955602152
6 840.2097456355932
7 2673.5331619142717
7 998.7181659465074
9 2980.1746195146584
8 1108.9464588375631
4 1438.7942981749893
9 1920.5376406573912
6 1968.112461185557
8 1424.755010255144
4 673.6259665253531
4 2915.9501941314693
8 552.8164128869712
9 1244.3563557527011
7 2934.994252653389
6 2270.7720136007742
7 491.932666372

D:\JetBrains\ANACONDA\envs\py37\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars


nan
nan
nan
nan
nan
迭代次数： 2
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
迭代次数： 3
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
迭代次数： 4
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
迭代次数： 5
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
迭代次数： 38
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
迭代次数： 39
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Optimal solution is: 6 2406.023117363554
